In [1]:
import torch
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from torch.autograd import Variable
from torch.autograd import grad
from nash_advreg import *

In [2]:
data = pd.read_csv("../data/winequality-white.csv", sep = ";")
X = data.loc[:, data.columns != "quality"]
y = data.quality
##
pca = PCA(n_components=X.shape[1], svd_solver='full')
pca.fit(X)
X = pca.fit_transform(X)
##

## Bayesian Case

In [15]:
MEAN = 0.01
VAR = 0.5
m = torch.distributions.Gamma(torch.tensor([MEAN**2/VAR]), torch.tensor([MEAN/VAR])) ## shape, rate

In [16]:
X_train, y_train, X_test, y_test = create_train_test(X,y)
params = {
                "epochs_rr"    : 1000,
                "lr_rr"        : 0.01,
                "lmb"          : 0.0,
                "c_d_train"    : torch.ones([len(y_train), 1]) * MEAN,
                "z_train"      : torch.zeros([len(y_train),1]),#.to("cuda"),
                "c_d_test"     : torch.ones([len(y_test), 1]) * MEAN,
                "z_test"       : torch.zeros([len(y_test),1]),#.to("cuda"),
                "outer_lr"     : 10e-6,
                "inner_lr"     : 0.0001,
                "outer_epochs" : 20,
                "inner_epochs" : 10,
                "n_samples"    : 10,
                "prior"        : m  
            }

In [17]:
c_d_train = params["prior"].sample(torch.Size([params["n_samples"], len(y_train)]))#.to("cuda")

In [18]:
w_bayes = train_bayes_rr_test(X_train, y_train, c_d_train, params, verbose = True)

epoch 0 weights tensor([[-1.2256, -0.2707, -0.9159,  0.3077, -0.1688, -0.3287,  0.1883,  1.2332,
         -0.8563,  1.3693,  1.2376,  0.1815]], grad_fn=<SubBackward0>)
epoch 1 weights tensor([[-1.1457, -0.2518, -0.8521,  0.2901, -0.1563, -0.3045,  0.1719,  1.1455,
         -0.7988,  1.2729,  1.1543,  0.5722]], grad_fn=<SubBackward0>)
epoch 2 weights tensor([[-1.0692, -0.2330, -0.7995,  0.2692, -0.1436, -0.2809,  0.1586,  1.0705,
         -0.7444,  1.1847,  1.0768,  0.9345]], grad_fn=<SubBackward0>)
epoch 3 weights tensor([[-0.9976, -0.2186, -0.7430,  0.2530, -0.1304, -0.2563,  0.1503,  0.9967,
         -0.6931,  1.1020,  1.0066,  1.2723]], grad_fn=<SubBackward0>)
epoch 4 weights tensor([[-0.9288, -0.2047, -0.6934,  0.2316, -0.1176, -0.2389,  0.1442,  0.9264,
         -0.6484,  1.0286,  0.9361,  1.5878]], grad_fn=<SubBackward0>)
epoch 5 weights tensor([[-0.8676, -0.1897, -0.6472,  0.2145, -0.1127, -0.2227,  0.1356,  0.8612,
         -0.6079,  0.9572,  0.8691,  1.8816]], grad_fn=<SubBack

KeyboardInterrupt: 

In [24]:
#torch.max(c_d_train)

In [25]:
#train_nash_rr_test(X_train, y_train, c_d_train, params, verbose = True)

In [26]:
#X_clean = X_train
#y = y_train
#w = torch.randn(1, X_clean.shape[1] + 1, requires_grad=True)
#c_d = torch.ones([len(y_train), 1]) * 1.0
#compute_backwar_derivative(X_clean, y, w, c_d, params)

# Test

In [27]:
w_rr = train_rr(X_train, y_train, params)
w_nash = train_nash_rr(X_train, y_train, params)
w_bayes = train_bayes_rr_test(X_train, y_train, c_d_train, params, verbose = True)

epoch 0 weights tensor([[-1.1243, -1.5460,  0.6223, -0.1238, -0.8885, -0.2759,  0.3897, -1.1944,
         -0.7417,  2.1541, -0.0957,  0.6892]], grad_fn=<SubBackward0>)
epoch 1 weights tensor([[-1.1083, -1.5193,  0.6112, -0.1194, -0.8737, -0.2718,  0.3832, -1.1750,
         -0.7316,  2.1192, -0.0954,  0.8772]], grad_fn=<SubBackward0>)
epoch 2 weights tensor([[-1.0888, -1.4886,  0.5989, -0.1148, -0.8556, -0.2668,  0.3752, -1.1521,
         -0.7182,  2.0780, -0.0945,  1.0635]], grad_fn=<SubBackward0>)
epoch 3 weights tensor([[-1.0665, -1.4533,  0.5836, -0.1103, -0.8371, -0.2606,  0.3676, -1.1252,
         -0.7045,  2.0326, -0.0917,  1.2473]], grad_fn=<SubBackward0>)
epoch 4 weights tensor([[-1.0406, -1.4136,  0.5705, -0.1052, -0.8153, -0.2540,  0.3605, -1.0976,
         -0.6881,  1.9798, -0.0892,  1.4311]], grad_fn=<SubBackward0>)
epoch 5 weights tensor([[-1.0128, -1.3692,  0.5525, -0.1000, -0.7917, -0.2461,  0.3500, -1.0667,
         -0.6699,  1.9200, -0.0878,  1.6130]], grad_fn=<SubBack

In [28]:
#X_clean = X_train.to("cuda")
#y = y_train.to("cuda")
#w = torch.randn(1, X_train.shape[1] + 1, requires_grad=True)#.to("cuda")
#compute_backward_derivative(X_clean, y, w, c_d_train[0], params)
#w_bayes = train_bayes_rr_test(X_train, y_train, c_d_train, params, verbose = True)

## Compare rmse's

In [32]:
c_d_test = params["prior"].sample(torch.Size([1, len(y_test)]))[0]
##
X_test_attacked = attack(X_test, w_rr, c_d_test, params["z_test"])
pred_attacked =  predict(X_test_attacked, w_rr)
pred_clean    =  predict(X_test, w_rr)
#
rmse_raw_clean = rmse( y_test, pred_clean )
rmse_raw_at    = rmse( y_test, pred_attacked )
#
##
X_test_attacked = attack(X_test, w_nash, c_d_test, params["z_test"])
pred_attacked =  predict(X_test_attacked, w_nash)
pred_clean    =  predict(X_test, w_nash)
#
rmse_nash_clean = rmse( y_test, pred_clean )
rmse_nash_at    = rmse( y_test, pred_attacked )
##
X_test_attacked = attack(X_test, w_bayes, c_d_test, params["z_test"])
pred_attacked =  predict(X_test_attacked, w_bayes)
pred_clean    =  predict(X_test, w_bayes)
#
rmse_bayes_clean = rmse( y_test, pred_clean )
rmse_bayes_at    = rmse( y_test, pred_attacked )

In [33]:
print("____Non-Strategic Defender____")
###
print( "Loss Clean test set: ", rmse_raw_clean )
###
print( "Loss attacked test set: ", rmse_raw_at )
###
print("\n____Strategic Bayes Defender____")
###
print( "Loss Bayes Clean test set: ", rmse_bayes_clean )
###
print( "Loss Bayes attacked test set: ", rmse_bayes_at )
###
print("\n____Strategic Nash Defender____")
###
print( "Loss Nash Clean test set: ", rmse_nash_clean )
###
print( "Loss Nash attacked test set: ", rmse_nash_at )
###


____Non-Strategic Defender____
Loss Clean test set:  tensor(0.7554, grad_fn=<SqrtBackward>)
Loss attacked test set:  tensor(1.0300, grad_fn=<SqrtBackward>)

____Strategic Bayes Defender____
Loss Bayes Clean test set:  tensor(2.1978, grad_fn=<SqrtBackward>)
Loss Bayes attacked test set:  tensor(3.0708, grad_fn=<SqrtBackward>)

____Strategic Nash Defender____
Loss Nash Clean test set:  tensor(5.9693, grad_fn=<SqrtBackward>)
Loss Nash attacked test set:  tensor(5.7955, grad_fn=<SqrtBackward>)


# Trash

In [42]:
"""
lr = params["outer_lr"]
T = params["outer_epochs"]
n_samples = params["n_samples"]
S = params["inner_epochs"]
ilr = params["inner_lr"]
X = torch.randn(X_clean.shape[0]*X_clean.shape[1], requires_grad=True).to("cuda")
##
Xt = torch.zeros(int(S), X.shape[0]).to("cuda")

print(Xt)
c_d = c_d_train[0]
gr = compute_backward_derivative(X_clean, X, Xt, y, w, c_d, S, ilr, params)
gr
print(Xt)
"""

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[-0.9592,  0.4510,  0.0282,  ...,  0.9187, -0.0400,  0.3336],
        [-0.9283,  0.4420,  0.0763,  ...,  0.8839, -0.0714,  0.3110],
        [-0.8979,  0.4352,  0.1225,  ...,  0.8542, -0.0977,  0.2921],
        ...,
        [ 0.5680,  0.5747,  2.1145,  ...,  0.3939, -0.3506,  0.0751],
        [ 0.5686,  0.5747,  2.1153,  ...,  0.3937, -0.3506,  0.0751],
        [ 0.5692,  0.5748,  2.1161,  ...,  0.3936, -0.3507,  0.0750]],
       device='cuda:0', grad_fn=<CopySlices>)


In [44]:
%%time
compute_backward_derivative(X_clean, X, Xt, y, w, c_d, S, ilr, params)

CPU times: user 1.48 s, sys: 88 ms, total: 1.57 s
Wall time: 1.56 s


tensor([[   70.1343, -1483.9263,   692.1609,  -201.3738,   850.2290,   -58.0689,
           -49.1796,   220.8937,   802.0889,   718.0339,   547.9590, -4548.6367]],
       device='cuda:0')

In [29]:
%%time
compute_backward_derivative(X_clean, y, w, c_d_train[0], params)

CPU times: user 1.29 s, sys: 36 ms, total: 1.33 s
Wall time: 1.5 s


tensor([[  991.3489,   467.4733,  -234.6497,  -610.4382,   644.6360,    -6.5245,
           380.2441,   386.2027,    54.4584,   218.5261,  -343.6595, -4406.8594]],
       device='cuda:0')

In [18]:
X_clean = X_train
y = y_train
##
lr = params["outer_lr"]
T = params["outer_epochs"]
n_samples = params["n_samples"]
w = torch.randn(1, X_clean.shape[1] + 1, requires_grad=True)

#X = torch.randn(X_clean.shape[0]*X_clean.shape[1], requires_grad=True)
#fm = lambda w, X: learner_cost_flatten(X, y, w, params)


grad = torch.zeros(1, X_clean.shape[1] + 1)
c_d = c_d_train[0].t()[0]
grad = compute_backwar_derivative(X_clean, y, w, c_d, params)